In [1]:
import gensim
import pandas as pd
import ast
import pandas as pd
import numpy as np

In [2]:
# df_lemmas = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/all_lemmas.csv', index_col = [0])

## Training on the whole dataset

In [ ]:
%%time

df_lemmas = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/all_lemmas.csv', index_col = [0])

df_lemmas['message_lemmas_list'] = df_lemmas['message_lemmas_list'].apply(ast.literal_eval)


In [ ]:
documents = df_lemmas['message_lemmas_list'].to_list()

In [ ]:
%%time
model = gensim.models.Word2Vec (documents, vector_size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

In [ ]:
model.save('/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec_correct.model')

In [14]:
%%time
model = gensim.models.Word2Vec (documents, vector_size=300, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

CPU times: user 4h 51min 33s, sys: 28min 31s, total: 5h 20min 5s
Wall time: 2h 12min 57s


(3319614815, 3940965080)

In [15]:
model.save('/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec_correct_300.model')

In [14]:
# model_ = gensim.models.Word2Vec.load('/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec.model')

In [19]:
w1 = "россиянин"
model.wv.most_similar(positive=w1, topn=30)

[('гражданин', 0.6260555982589722),
 ('иностранец', 0.6161984801292419),
 ('респондент', 0.5866599082946777),
 ('согражданин', 0.5792011022567749),
 ('москвич', 0.5500414967536926),
 ('турист', 0.5197361707687378),
 ('украинец', 0.5189060568809509),
 ('человек', 0.5046504139900208),
 ('британец', 0.4958539605140686),
 ('казахстанец', 0.49145030975341797),
 ('американец', 0.48877468705177307),
 ('соотечественник', 0.47792723774909973),
 ('петербуржец', 0.4773894250392914),
 ('мигрант', 0.4758749008178711),
 ('европеец', 0.4724271893501282),
 ('белорус', 0.4628015458583832),
 ('население', 0.4620310366153717),
 ('житель', 0.4598851501941681),
 ('россиянка', 0.4535672664642334),
 ('пользователь', 0.4482652246952057),
 ('опросить', 0.44760861992836),
 ('путешественник', 0.44352179765701294),
 ('россия', 0.431467741727829),
 ('крымчанин', 0.42702823877334595),
 ('афганец', 0.42592233419418335),
 ('горожанин', 0.42527905106544495),
 ('русских', 0.4248400628566742),
 ('беженец', 0.42481029033

In [5]:
# # from gensim.models import Word2Vec
# #
#
# # model_path = '/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec.model'
# # model = Word2Vec.load(model_path)
#
# vocabulary = list(model.wv.index_to_key)
#
# print(vocabulary)

## Averaging the morphological forms

Here I find all the morphological forms of the most popular labels. I additionaly find the misspelled versions (excluding those that are in fact real different words).

In [71]:
from gensim.models import Word2Vec
model_path = '/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec_correct.model'
model = Word2Vec.load(model_path)

In [72]:
w1 = 'хохлушка'
model.wv.most_similar(positive=w1, topn=30)

[('нацистка', 0.6061878800392151),
 ('шалава', 0.6014500260353088),
 ('шмара', 0.5997158288955688),
 ('бабища', 0.5862389802932739),
 ('бандеровка', 0.580834150314331),
 ('мерзавка', 0.5768174529075623),
 ('дамочка', 0.5684010982513428),
 ('украинка', 0.5600405335426331),
 ('дивчина', 0.5577738285064697),
 ('карлица', 0.5557477474212646),
 ('бабёнка', 0.5546216368675232),
 ('девка', 0.5523253679275513),
 ('хабалка', 0.5515905618667603),
 ('деваха', 0.5451499223709106),
 ('стерва', 0.5445281863212585),
 ('одесситка', 0.5409867167472839),
 ('алкоголичка', 0.5364329814910889),
 ('жинка', 0.5354903340339661),
 ('свинособака', 0.5255479216575623),
 ('пропагандистка', 0.525350034236908),
 ('ципсота', 0.5225162506103516),
 ('свинособак', 0.5224473476409912),
 ('шлюха', 0.5222390294075012),
 ('танька', 0.5213223099708557),
 ('парубок', 0.5197714567184448),
 ('юлька', 0.5162315368652344),
 ('истеричка', 0.5158720016479492),
 ('полуграмотная', 0.5151511430740356),
 ('еврейка', 0.5148496031761169

In [73]:
popular_label_list = ['укр',
                      'украинец',
                      'украинка',
                      'хохол',
                      'хохлушка',
                      'укроп',
                      'Украина',
                      'украинский',
                      'украинская',
                      'украинское',
                      'хохлятский',
                      'хохлятская',
                      'хохлятское',
                      'укропский', #укровский сюди ж піде в рамках місспеллінга
                      'укропская',
                      'укропское']

In [74]:
import pymorphy2

def get_word_forms(word):
    morph = pymorphy2.MorphAnalyzer()
    parsed_word = morph.parse(word)
    forms = set()
    for parse_result in parsed_word:
        for form in parse_result.lexeme:
            forms.add(form.word)
    return forms

In [75]:
all_forms_dict = {}
for word in popular_label_list:
    all_forms_dict[word]=get_word_forms(word)

In [76]:
all_forms_dict

{'укр': {'укр'},
 'украинец': {'украинец',
  'украинца',
  'украинцам',
  'украинцами',
  'украинцах',
  'украинце',
  'украинцев',
  'украинцем',
  'украинцу',
  'украинцы'},
 'украинка': {'украинка',
  'украинкам',
  'украинками',
  'украинках',
  'украинке',
  'украинки',
  'украинкой',
  'украинкою',
  'украинку',
  'украинок'},
 'хохол': {'хохла',
  'хохлам',
  'хохлами',
  'хохлах',
  'хохле',
  'хохлов',
  'хохлом',
  'хохлу',
  'хохлы',
  'хохол'},
 'хохлушка': {'хохлушек',
  'хохлушка',
  'хохлушкам',
  'хохлушками',
  'хохлушках',
  'хохлушке',
  'хохлушки',
  'хохлушкой',
  'хохлушкою',
  'хохлушку'},
 'укроп': {'укроп',
  'укропа',
  'укропам',
  'укропами',
  'укропах',
  'укропе',
  'укропов',
  'укропом',
  'укропу',
  'укропы'},
 'Украина': {'украина',
  'украине',
  'украиной',
  'украиною',
  'украину',
  'украины'},
 'украинский': {'украинская',
  'украинские',
  'украинский',
  'украинским',
  'украинскими',
  'украинских',
  'украинского',
  'украинское',
  'украин

In [77]:
target_word = "хохлушка"
max_distance = 1

# misspelled_versions = find_misspelled_words(model, target_word, max_distance)

In [78]:
vocabulary = list(model.wv.index_to_key)

In [79]:
len(vocabulary)

879663

In [80]:
def is_misspelled(word):
    morph = pymorphy2.MorphAnalyzer()
    parsed_word = morph.parse(word)
    for parse_result in parsed_word:
        if parse_result.tag.POS is not None:
            return False
    return True

In [81]:
word_to_check = "хxол"
is_word_misspelled = is_misspelled(word_to_check)
if is_word_misspelled:
    print(f"The word '{word_to_check}' is a misspelling.")
else:
    print(f"The word '{word_to_check}' is a real word.")

The word 'хxол' is a real word.


Neither pyspellchecker nor pymorphy2 is able to reliably tell the difference between the real and misspelled word (the example is above), so we'll look trough the possible misspellings (Levenstein distance 1 for every inflected form) and relying on our Russian language knowledge select the misspelled versions. Those will be considered for inclusion in the general list of morphological forms.

In [82]:
import Levenshtein

def find_misspelled_words(model, target_word, max_distance):
    misspelled_words = []
    for word in model.wv.key_to_index:
        if abs(len(word) - len(target_word)) <= max_distance:
            distance = Levenshtein.distance(target_word, word)
            if distance <= max_distance:
                misspelled_words.append(word)

    return misspelled_words


In [83]:
target_word = "хохлушка"
max_distance = 1

misspelled_versions = find_misspelled_words(model, target_word, max_distance)
misspelled_versions

['хохлушка', 'хохлуша', 'хохлушкам']

In [84]:
# We're creating the list with all the valid inflections to check misspelling against to

all_inflection_list = [all_forms_dict[word] for word in popular_label_list]
all_inflection_list = [item for d in all_inflection_list for item in d]

In [85]:
misspellings_dict = {}
for word in all_forms_dict:
    inflected_word_list = list(all_forms_dict[word])
    all_misspellings_for_word = []
    for word_form in inflected_word_list:
        misspellings = find_misspelled_words(model, word_form, max_distance)
        misspellings = [misspelled_form for misspelled_form in misspellings if misspelled_form not in all_inflection_list]
        all_misspellings_for_word.append(misspellings)
    all_misspellings_for_word = [item for sublist in all_misspellings_for_word for item in sublist]

    misspellings_dict[word]=all_misspellings_for_word


In [86]:
misspellings_to_remove = {'укр': ['ук', 'скр', 'укры', 'нкр', 'окр', 'кр', 'пкр', 'ур', 'акр', 'укор', 'мкр', 'уур', 'унр', 'укро','угр','укс','укрф', 'уяр','укра','дкр','фкр','ркр','вкр','уку','укв','упр','уки','цкр','укре','зкр','гкр','укз','уор','ука','утр','ткр','укн','уктр','укб','убр','укх','увр','гукр','урр','умр','икр','уэр','удр','укт', 'укп','укм','уко','укнр','ушр','укрл','укрп','екр','ухр','бкр','шкр','уир','укрв'],
                          'украинец': ['украинск', 'украинам','украинру'],

                          'украинка': ['украинах', 'украинких', 'украинкая', 'украинск', 'украинои','украинкую', 'укранкою','украинкую', 'украинру', 'украински', 'украини', 'украинои', 'украиныи', 'украинких', 'украинкие', 'украинуи', 'украиской', 'украиныой', 'украиснкой', 'украинуой','украинуой', 'украинске', 'украинкие','украинам', 'украинкая','украинкая','украинска', 'украинеа','украинами'],

                          'хохол': ['хохот','хохо','хорол','бохол','хохон','хохламы','хохмам','хозлам','хохму','хохма','хотла','хоха','сохла', 'хойла','хола','холла','хоула','хохловя','хохлову','рохлов','хихлов','хозлов','хохлотв','дхохлов','хохлово','хохлому','хохлома','хохломы','хохло','хойлом','хохломе','хохмы','холлы','хозлы','хорлы', 'хохлыс','холы','холе','холле','хохуле','хохе','хохе'],

                          'хохлушка': [],

                          'укроп':['укрепов','укропол','укропоп','укроптв','укреп','укрой','кроп','укропв','укрокп','укрою','укрп','укрепам','укроспам','укрепом','укропол','укропоп','укрепе','укропв','укрепа','укропв','укрепы','укрепами','укрепах','укрепу'],

                          'Украина':['украиныс','украинц','украинв','украицы','окраины','уокраины','окраиной','краиной','украну','краину','украинц','украинру','украинуи','украиру','украинут','окраина','краина','украинц','укрвина','укранна','укрпина','краине','украинев','украинц','окраине', 'украинес'],

                          'украинский':[],

                          'украинская': [],

                          'украинское': [],

                          'хохлятский': [],

                          'хохлятская': [],

                          'хохлятское': [],

                          'укропский': [],

                          'укропская': [],

                          'укропское': []
                          }


The unfortunate downside is that for some words misspellings look more like just the other word, so validity of the choice to remove is decided on the level of the word, not the general list.

It could be beneficial to include also украино, хохло, укро.

In [87]:
clean_misspellings_dict = {}
for word in popular_label_list:
    misspells = [word_ for word_ in misspellings_dict[word] if word_ not in misspellings_to_remove[word]]
    clean_misspellings_dict[word] = misspells

In [88]:
clean_misspellings_dict

{'укр': ['укру', 'укри'],
 'украинец': ['украинци',
  'украинц',
  'украинеа',
  'яукраинца',
  'украинци',
  'украинц',
  'кукраинцам',
  'усраинцам',
  'украинцям',
  'укранцем',
  'украинцям',
  'украинах',
  'украинци',
  'окраинцы',
  'яукраинцы',
  'украинц',
  'украдинцы',
  'украицы',
  'усраинцы',
  'укроинцы',
  'украйнцы',
  'краинцы',
  'окраинцами',
  'сукраинцами',
  'украинами',
  'оукраинцами',
  'украинцева',
  'иукраинцев',
  'украинцеву',
  'украинев',
  'уукраинцев',
  'укранцев',
  'украинцевъ',
  'украицев',
  'усраинцев',
  'окраинцев',
  'украиснцев',
  'уараинцев',
  'украинцевс',
  'украиньцев',
  'украинци',
  'украинц',
  'украинеи',
  'украинев',
  'украинець',
  'украинц',
  'вукраинец',
  'укранец',
  'окраинец',
  'украинес',
  'иукраинец',
  'украинеа',
  'украиниц'],
 'украинка': ['укранка',
  'краинка',
  'украинци',
  'украинеи',
  'укранки',
  'укринки',
  'украино',
  'укранок'],
 'хохол': ['хохл',
  'хохли',
  'хохлв',
  'хохола',
  'ххла',
  'хох

In [89]:
all_forms_dict

{'укр': {'укр'},
 'украинец': {'украинец',
  'украинца',
  'украинцам',
  'украинцами',
  'украинцах',
  'украинце',
  'украинцев',
  'украинцем',
  'украинцу',
  'украинцы'},
 'украинка': {'украинка',
  'украинкам',
  'украинками',
  'украинках',
  'украинке',
  'украинки',
  'украинкой',
  'украинкою',
  'украинку',
  'украинок'},
 'хохол': {'хохла',
  'хохлам',
  'хохлами',
  'хохлах',
  'хохле',
  'хохлов',
  'хохлом',
  'хохлу',
  'хохлы',
  'хохол'},
 'хохлушка': {'хохлушек',
  'хохлушка',
  'хохлушкам',
  'хохлушками',
  'хохлушках',
  'хохлушке',
  'хохлушки',
  'хохлушкой',
  'хохлушкою',
  'хохлушку'},
 'укроп': {'укроп',
  'укропа',
  'укропам',
  'укропами',
  'укропах',
  'укропе',
  'укропов',
  'укропом',
  'укропу',
  'укропы'},
 'Украина': {'украина',
  'украине',
  'украиной',
  'украиною',
  'украину',
  'украины'},
 'украинский': {'украинская',
  'украинские',
  'украинский',
  'украинским',
  'украинскими',
  'украинских',
  'украинского',
  'украинское',
  'украин

In [90]:
df_ = pd.DataFrame([all_forms_dict])
df_.to_json('all_forms_dict.json',orient='records', lines=True)

In [91]:
word = 'ххлов'

if word in model.wv.key_to_index:
    word_vector = model.wv[word]
    print(f"The vector for the word '{word}':\n{word_vector}")
else:
    print(f"The word '{word}' is not in the vocabulary.")

The vector for the word 'ххлов':
[-0.43288842  0.7436105  -0.09290253 -0.02120434  0.02870027 -0.13131145
  0.11315405  0.03146933  0.19433673 -0.12791622  0.03809118 -0.29372498
  0.27211308 -0.27973303 -0.412195   -0.03709674 -0.11171024  0.05461736
 -0.27791902  0.16175213  0.07480086 -0.00944865 -0.01333196  0.3723256
  0.60280013  0.0032816   0.0651744  -0.32838944 -0.23008314 -0.5041946
 -0.23298502 -0.23291504  0.5789121  -0.05833485 -0.2310122   0.01809216
 -0.26653925  0.6568714  -0.17577997 -0.15913822  0.02746626 -0.12497678
 -0.05738966 -0.02048556  0.13583952 -0.22931214 -0.37275848 -0.17432179
 -0.2166473   0.3330127   0.00532212  0.08767729 -0.0558027   0.22079192
  0.13893016  0.23004353  0.10277361 -0.38730282  0.07255914  0.08250796
 -0.06671572  0.39566067  0.3268311   0.00227097  0.00352522 -0.20208976
 -0.29345524 -0.23287545 -0.31982905 -0.31867588 -0.2147223  -0.34511644
  0.39490074 -0.37337115  0.01496766  0.2954205   0.21198319  0.21654636
 -0.47028583 -0.3498

In [92]:
final_forms_list = {}
for word in popular_label_list:
    final_forms_list[word]= list(all_forms_dict[word])+clean_misspellings_dict[word]

In [93]:
final_forms_list

{'укр': ['укр', 'укру', 'укри'],
 'украинец': ['украинца',
  'украинце',
  'украинцам',
  'украинцем',
  'украинцах',
  'украинцы',
  'украинцами',
  'украинцев',
  'украинцу',
  'украинец',
  'украинци',
  'украинц',
  'украинеа',
  'яукраинца',
  'украинци',
  'украинц',
  'кукраинцам',
  'усраинцам',
  'украинцям',
  'укранцем',
  'украинцям',
  'украинах',
  'украинци',
  'окраинцы',
  'яукраинцы',
  'украинц',
  'украдинцы',
  'украицы',
  'усраинцы',
  'укроинцы',
  'украйнцы',
  'краинцы',
  'окраинцами',
  'сукраинцами',
  'украинами',
  'оукраинцами',
  'украинцева',
  'иукраинцев',
  'украинцеву',
  'украинев',
  'уукраинцев',
  'укранцев',
  'украинцевъ',
  'украицев',
  'усраинцев',
  'окраинцев',
  'украиснцев',
  'уараинцев',
  'украинцевс',
  'украиньцев',
  'украинци',
  'украинц',
  'украинеи',
  'украинев',
  'украинець',
  'украинц',
  'вукраинец',
  'укранец',
  'окраинец',
  'украинес',
  'иукраинец',
  'украинеа',
  'украиниц'],
 'украинка': ['украинка',
  'украин

In [94]:
%%time

vectors_dict = {}
for word in popular_label_list:
    word_vec_list = [model.wv[form] for form in final_forms_list[word] if form in vocabulary]
    vectors_dict[word] = word_vec_list

CPU times: user 2.62 s, sys: 24.3 ms, total: 2.64 s
Wall time: 2.64 s


In [95]:
%%time

centroids_dict = {}
for word in popular_label_list:
    centroid_array=np.array(vectors_dict[word])
    centroid = np.mean(centroid_array, axis=0)
    centroids_dict[word] = centroid

CPU times: user 895 µs, sys: 850 µs, total: 1.75 ms
Wall time: 1.27 ms


In [96]:
centroids_dict['украинск_'] = np.mean(np.vstack([centroids_dict['украинский'],centroids_dict['украинская'],centroids_dict['украинское']]), axis=0)

In [97]:
centroids_dict['хохлятск_'] = np.mean(np.vstack([centroids_dict['хохлятский'],centroids_dict['хохлятская'],centroids_dict['хохлятское']]), axis=0)

In [98]:
centroids_dict['укропск_'] = np.mean(np.vstack([centroids_dict['укропский'],centroids_dict['укропская'],centroids_dict['укропское']]), axis=0)

In [99]:
centroids_dict['украин_'] = np.mean(np.vstack([centroids_dict['украинец'],centroids_dict['украинка']]), axis=0)

In [100]:
centroids_dict['хох_'] = np.mean(np.vstack([centroids_dict['хохол'],centroids_dict['хохлушка']]), axis=0)

In [101]:
labels_to_rm = ['украинский', 'украинская', 'украинское', 'хохлятский', 'хохлятская', 'хохлятское', 'укропский', 'укропская', 'укропское']

for label in labels_to_rm:
    del centroids_dict[label]

In [102]:
centroids_dict.keys()

dict_keys(['укр', 'украинец', 'украинка', 'хохол', 'хохлушка', 'укроп', 'Украина', 'украинск_', 'хохлятск_', 'укропск_', 'украин_', 'хох_'])

In [105]:
centroids_dict

{'укр': array([-0.42878458, -0.14173278,  0.35514474,  0.30835035,  0.46497726,
        -1.2164317 ,  0.58628994, -0.12765102,  0.3588579 , -0.4673873 ,
         0.12715448, -0.16914482, -0.0426648 ,  0.6434408 ,  0.35352626,
         0.2988735 ,  0.04927415,  0.3444352 ,  0.4261329 , -0.20419282,
         0.04479333,  1.4654046 , -0.5292267 ,  0.28453726, -0.39278516,
        -0.18567061,  0.17548023, -0.86843807,  1.0515684 ,  0.12538476,
        -0.7291705 , -0.43656096, -0.81799984, -0.25885177, -0.43331102,
         0.8611107 , -0.3890923 ,  0.33946648, -0.02101213,  0.4540206 ,
         0.25284043,  0.10048433, -0.28265926, -0.15195934,  0.59896463,
        -0.58201647, -0.68531245, -0.1788721 , -0.03205414,  0.2925841 ,
        -0.05591347, -0.9945833 , -0.29373315, -0.5213817 , -1.3020766 ,
         0.3862245 ,  0.10938714, -0.42029762,  1.1768816 , -0.60302633,
         0.9622045 , -0.01683349, -0.7709419 ,  1.3404994 ,  0.07809808,
        -0.6822796 , -0.44278026, -0.5837576

In [107]:
import numpy as np

def save_dictionary_to_file(dictionary, file_name):
    np.savez(file_name, **dictionary)

In [108]:
save_dictionary_to_file(centroids_dict, 'centroids_dict.npz')

In [109]:
# import numpy as np
#
# def load_dictionary_from_file(file_name):
#     with np.load(file_name) as data:
#         return {key: data[key] for key in data}

In [111]:
# loaded_dict = load_dictionary_from_file('centroids_dict.npz')

### Training on the 3-month timesteps

In [2]:
import pandas as pd
import ast
import numpy as np
from gensim.models import Word2Vec
from datetime import timedelta

In [2]:
%%time
df_lemmas_all_time = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/data/all_lemmas_w_dates.csv', index_col=[0])

CPU times: user 2min 52s, sys: 1min 16s, total: 4min 8s
Wall time: 7min 57s


In [5]:
# We need to parse lists correctly but in the way that's affordable in terms of memory
chunk_size = 5000
def process_chunk(chunk):
    chunk['message_lemmas_list'] = chunk['message_lemmas_list'].apply(ast.literal_eval)
    return chunk
chunks = []

In [4]:
%%time

for start in range(0, len(df_lemmas_all_time), chunk_size):
    end = start + chunk_size
    chunk = df_lemmas_all_time.iloc[start:end].copy()
    processed_chunk = process_chunk(chunk)
    chunks.append(processed_chunk)

df_lemmas_all_time = pd.concat(chunks, axis=0)

CPU times: user 6min 58s, sys: 4min 34s, total: 11min 32s
Wall time: 29min 26s


In [6]:
%%time
df_lemmas_all_time['date_formatted'] = pd.to_datetime(df_lemmas_all_time['date_formatted'])

CPU times: user 2.43 s, sys: 20.8 s, total: 23.2 s
Wall time: 1min 41s


In [7]:
def divide_into_timesteps(df, timedelta_months=3):
    min_date = df['date_formatted'].min()
    max_date = df['date_formatted'].max()
    time_step_limit = min_date + pd.DateOffset(months=timedelta_months)

    time_slices = []
    while time_step_limit <= max_date:
        time_slice = df[(df['date_formatted'] >= min_date) & (df['date_formatted'] < time_step_limit)]
        time_slices.append(time_slice)
        min_date = time_step_limit
        time_step_limit = time_step_limit + pd.DateOffset(months=timedelta_months)

    if min_date < max_date:
        time_slice = df[df['date_formatted'] >= min_date]
        time_slices.append(time_slice)

    return time_slices

In [8]:
%%time
time_slices = divide_into_timesteps(df_lemmas_all_time)

CPU times: user 3.72 s, sys: 54.4 s, total: 58.1 s
Wall time: 4min 32s


In [9]:
len(time_slices)

30

In [12]:
time_slices[0]

,id,date_formatted,message,message_final,channel_name,message_lemmas_list
235734,150.0,2015-12-22 06:00:32+00:00,Доброе утро! Самое важное к этому часу Ракет...,доброе утро самое важное к этому часу ракета...,tvrain,"[добрый, утро, , самое, важный, к, этот, час,..."
235735,149.0,2015-12-21 06:03:00+00:00,Доброе утро! Главные новости к этому часу: ...,доброе утро главные новости к этому часу в к...,tvrain,"[добрый, утро, , главный, новость, к, этот, ч..."
235736,148.0,2015-12-20 08:05:28+00:00,Хорошего вам воскресенья! Вот главные новости ...,хорошего вам воскресенья вот главные новости к...,tvrain,"[хороший, вам, воскресение, вот, главный, ново..."
235737,147.0,2015-12-19 08:13:32+00:00,Доброе утро! Вот главные новости к этому часу:...,доброе утро вот главные новости к этому часу ...,tvrain,"[добрый, утро, вот, главный, новость, к, этот,..."
235738,146.0,2015-12-18 06:08:16+00:00,Доброе утро! Вот главные новости: Reuters у...,доброе утро вот главные новости узнало о с...,tvrain,"[добрый, утро, вот, главный, новость, , уз..."
...,...,...,...,...,...,...
5638168,10.0,2015-09-28 12:44:55+00:00,«Форд» будет производиться в Калининграде http...,форд будет производиться вкалининграде,izvestia,"[форд, будет, производиться, вкалининграде]"
5638169,9.0,2015-09-28 09:14:53+00:00,В президентский авиапарк купят еще два командн...,в президентский авиапарк купят еще два командн...,izvestia,"[в, президентский, авиапарк, купить, еще, два,..."
5638170,8.0,2015-09-24 13:48:28+00:00,«В 2015 году продажи российского кино за рубеж...,в году продажи российского кино за рубеж сост...,izvestia,"[в, , год, продажа, российский, кино, за, руб..."
5638171,7.0,2015-09-24 11:40:51+00:00,Более 80% россиян против повышения пенсионного...,более россиян против повышения пенсионного во...,izvestia,"[более, , россиянин, против, повышение, пенси..."


In [3]:
model_path = "/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/final_models/full_dataset_word2vec_correct.model"
gensim_model = Word2Vec.load(model_path)

In [2]:
# for i, df in enumerate(time_slices):
#     df.to_csv(f'/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/time_step_dataframes/dataframe_{i}.csv', index=False)

In [ ]:
# import copy
#
# def train_word2vec_models(base_model, time_slices):
#     epochs = 20
#     timestep_models = []
#
#     for time_slice in time_slices:
#         iter_model = copy.deepcopy(base_model)
#         documents = time_slice['message_lemmas_list'].to_list()
#
#         iter_model.build_vocab(documents, update=True)
#         iter_model.train(documents,
#                          total_examples=iter_model.corpus_count,
#                          epochs=epochs)
#         timestep_models.append(iter_model)
#
#     return timestep_models

In [11]:
# import os
#
# slices_directory = '/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/time_step_dataframes'
#
# for i, time_slice in enumerate(time_slices):
#     if i<27:
#         continue
#     else:
#         start_date = time_slice['date_formatted'].min().strftime("%Y-%m-%d")
#         end_date = time_slice['date_formatted'].max().strftime("%Y-%m-%d")
#         slice_name = f"df_time_slice_{i+1}_{start_date}_to_{end_date}.csv"
#         slice_path = os.path.join(slices_directory, slice_name)
#         time_slice.to_csv(slice_path)
#         print(f"Done with {slice_name}")

Done with df_time_slice_28_2022-06-22_to_2022-09-22.csv
Done with df_time_slice_29_2022-09-22_to_2022-12-22.csv
Done with df_time_slice_30_2022-12-22_to_2022-12-26.csv


In [4]:
import os
import pandas as pd

slices_directory = '/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/time_step_dataframes'
time_slices = []
file_names = os.listdir(slices_directory)

file_names = sorted(file_names, key=lambda name: int(name.split('_')[3]) if name.endswith('.csv') else float('inf'))


In [5]:
print(file_names)

['df_time_slice_1_2015-09-22_to_2015-12-22.csv', 'df_time_slice_2_2015-12-23_to_2016-03-22.csv', 'df_time_slice_3_2016-03-23_to_2016-06-22.csv', 'df_time_slice_4_2016-06-22_to_2016-09-22.csv', 'df_time_slice_5_2016-09-22_to_2016-12-22.csv', 'df_time_slice_6_2016-12-22_to_2017-03-22.csv', 'df_time_slice_7_2017-03-22_to_2017-06-22.csv', 'df_time_slice_8_2017-06-22_to_2017-09-22.csv', 'df_time_slice_9_2017-09-22_to_2017-12-22.csv', 'df_time_slice_10_2017-12-22_to_2018-03-22.csv', 'df_time_slice_11_2018-03-22_to_2018-06-22.csv', 'df_time_slice_12_2018-06-22_to_2018-09-22.csv', 'df_time_slice_13_2018-09-22_to_2018-12-22.csv', 'df_time_slice_14_2018-12-22_to_2019-03-22.csv', 'df_time_slice_15_2019-03-22_to_2019-06-22.csv', 'df_time_slice_16_2019-06-22_to_2019-09-22.csv', 'df_time_slice_17_2019-09-22_to_2019-12-22.csv', 'df_time_slice_18_2019-12-22_to_2020-03-22.csv', 'df_time_slice_19_2020-03-22_to_2020-06-22.csv', 'df_time_slice_20_2020-06-22_to_2020-09-22.csv', 'df_time_slice_21_2020-09-22

In [6]:
%%time
for file_name in file_names:
    if file_name.endswith('.csv'):
        file_path = os.path.join(slices_directory, file_name)
        df = pd.read_csv(file_path, index_col=0)
        df['date_formatted'] = pd.to_datetime(df['date_formatted'])
        # print(type(df.date_formatted.iloc[0]))
        time_slices.append(df)

print("Loaded all time_slices dataframes into the list.")

Loaded all time_slices dataframes into the list.
CPU times: user 2min 46s, sys: 27.7 s, total: 3min 14s
Wall time: 3min 40s


In [7]:
# type(time_slices[0].date_formatted.iloc[0])

In [12]:
import copy
def train_word2vec_models(base_model, time_slices, models_directory):
    epochs = 20
    timestep_models = []

    for i, time_slice in enumerate(time_slices):
        if i<26:
            continue
        else:
            iter_model = copy.deepcopy(base_model)
            documents = time_slice['message_lemmas_list'].to_list()

            iter_model.build_vocab(documents, update=True)
            iter_model.train(documents,
                             total_examples=iter_model.corpus_count,
                             epochs=epochs)

            start_date = time_slice['date_formatted'].min().strftime("%Y-%m-%d")
            end_date = time_slice['date_formatted'].max().strftime("%Y-%m-%d")
            model_name = f"word2vec_time_slice_{i+1}_{start_date}_to_{end_date}.model"
            model_path = os.path.join(models_directory, model_name)
            iter_model.save(model_path)

            timestep_models.append(iter_model)
            print(f"Done with {model_name}")

    return timestep_models


In [13]:
# import copy
# def train_word2vec_models(base_model, time_slices, models_directory):
#     epochs = 20
#     timestep_models = []
#
#     for i, time_slice in enumerate(time_slices):
#         iter_model = copy.deepcopy(base_model)
#         documents = time_slice['message_lemmas_list'].to_list()
#
#         iter_model.build_vocab(documents, update=True)
#         iter_model.train(documents,
#                          total_examples=iter_model.corpus_count,
#                          epochs=epochs)
#
#         start_date = time_slice['date_formatted'].min().strftime("%Y-%m-%d")
#         end_date = time_slice['date_formatted'].max().strftime("%Y-%m-%d")
#         model_name = f"word2vec_time_slice_{i+1}_{start_date}_to_{end_date}.model"
#         model_path = os.path.join(models_directory, model_name)
#         iter_model.save(model_path)
#
#         timestep_models.append(iter_model)
#         print(f"Done with {model_name}")
#
#     return timestep_models


In [14]:
import os
from gensim.models import Word2Vec

In [15]:
models_directory = '/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/models_in_time_more_ep'
timestep_models = train_word2vec_models(gensim_model, time_slices, models_directory)

Done with word2vec_time_slice_27_2022-03-22_to_2022-06-22.model
Done with word2vec_time_slice_28_2022-06-22_to_2022-09-22.model
Done with word2vec_time_slice_29_2022-09-22_to_2022-12-22.model
Done with word2vec_time_slice_30_2022-12-22_to_2022-12-26.model


## Reading models and calculating the nearest N

In [12]:
import os
from gensim.models import Word2Vec
from typing import List

In [53]:
# %%time
#
# models_directory = "/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/models_in_time"
# model_filenames = [f for f in os.listdir(models_directory) if f.endswith(".model")]
#
# models = []
# for filename in model_filenames:
#     model_path = os.path.join(models_directory, filename)
#     models.append(Word2Vec.load(model_path))

In [54]:
# def find_closest_words(models: List[Word2Vec], given_vector, topn: int = 100):
#     all_closest_words = []
#     for model in models:
#         closest_words = model.wv.similar_by_vector(given_vector, topn=topn)
#         all_closest_words.extend(closest_words)
#     return all_closest_words

In [5]:
def find_closest_words_for_single_model(model_path: str, given_vector, topn: int = 10):
    try:
        model = Word2Vec.load(model_path)
        closest_words = model.wv.similar_by_vector(given_vector, topn=topn)
        del model
        return closest_words
    except Exception as e:
        print(f"Error processing model {model_path}: {e}")
        return []

In [56]:
models_directory = "/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/models_in_time"
model_filenames = [f for f in os.listdir(models_directory) if f.endswith(".model")]

In [57]:
centroids_dict['укр']

array([-0.42878458, -0.14173278,  0.35514474,  0.30835035,  0.46497726,
       -1.2164317 ,  0.58628994, -0.12765102,  0.3588579 , -0.4673873 ,
        0.12715448, -0.16914482, -0.0426648 ,  0.6434408 ,  0.35352626,
        0.2988735 ,  0.04927415,  0.3444352 ,  0.4261329 , -0.20419282,
        0.04479333,  1.4654046 , -0.5292267 ,  0.28453726, -0.39278516,
       -0.18567061,  0.17548023, -0.86843807,  1.0515684 ,  0.12538476,
       -0.7291705 , -0.43656096, -0.81799984, -0.25885177, -0.43331102,
        0.8611107 , -0.3890923 ,  0.33946648, -0.02101213,  0.4540206 ,
        0.25284043,  0.10048433, -0.28265926, -0.15195934,  0.59896463,
       -0.58201647, -0.68531245, -0.1788721 , -0.03205414,  0.2925841 ,
       -0.05591347, -0.9945833 , -0.29373315, -0.5213817 , -1.3020766 ,
        0.3862245 ,  0.10938714, -0.42029762,  1.1768816 , -0.60302633,
        0.9622045 , -0.01683349, -0.7709419 ,  1.3404994 ,  0.07809808,
       -0.6822796 , -0.44278026, -0.5837576 , -1.0209492 , -0.62

In [58]:
def get_all_nearest_wordvecs(key, model_filenames=model_filenames, topn=10):
    all_closest_words = []

    for filename in model_filenames:
        model_path = os.path.join(models_directory, filename)
        closest_words = find_closest_words_for_single_model(model_path, key, topn=topn)
        all_closest_words.extend(closest_words)
        # print(f'Done with model {filename}')

    unique_closest_words = list(dict(all_closest_words).items())

    return unique_closest_words


In [59]:
%%time

nearest_ukr = get_all_nearest_wordvecs(centroids_dict['укр'])

CPU times: user 1min 52s, sys: 36.3 s, total: 2min 29s
Wall time: 1min 57s


In [60]:
# closest_words = find_closest_words_for_single_model(model_path, centroids_dict['укр'], topn=100)

In [61]:
nearest_ukr

[('укр', 0.977008044719696),
 ('укро', 0.6340225338935852),
 ('насмикав', 0.5467818975448608),
 ('зрадная', 0.5411729216575623),
 ('кляти', 0.524753987789154),
 ('мюмзики', 0.5390231013298035),
 ('укроканалы', 0.526896595954895),
 ('цханоская', 0.5222957730293274),
 ('венерична', 0.5210613012313843),
 ('ефрв', 0.5089078545570374),
 ('мыкола', 0.5238763093948364),
 ('шароварный', 0.512841522693634),
 ('укрорейха', 0.5100848078727722),
 ('хохлы', 0.5254834890365601),
 ('хохол', 0.5190120339393616),
 ('щирый', 0.5069569945335388),
 ('укропской', 0.518076479434967)]

In [62]:
len(nearest_ukr)

17

In [63]:
%%time
nearest_vals_dict = {}
for key in centroids_dict.keys():
    nearest_vals_dict[key] = get_all_nearest_wordvecs(centroids_dict[key])

CPU times: user 23min 49s, sys: 5min 53s, total: 29min 42s
Wall time: 22min 21s


In [64]:
nearest_vals_dict

{'укр': [('укр', 0.977008044719696),
  ('укро', 0.6340225338935852),
  ('насмикав', 0.5467818975448608),
  ('зрадная', 0.5411729216575623),
  ('кляти', 0.524753987789154),
  ('мюмзики', 0.5390231013298035),
  ('укроканалы', 0.526896595954895),
  ('цханоская', 0.5222957730293274),
  ('венерична', 0.5210613012313843),
  ('ефрв', 0.5089078545570374),
  ('мыкола', 0.5238763093948364),
  ('шароварный', 0.512841522693634),
  ('укрорейха', 0.5100848078727722),
  ('хохлы', 0.5254834890365601),
  ('хохол', 0.5190120339393616),
  ('щирый', 0.5069569945335388),
  ('укропской', 0.518076479434967)],
 'украинец': [('украинец', 0.7451037764549255),
  ('украинци', 0.6607651114463806),
  ('киевлянин', 0.6365122199058533),
  ('одессит', 0.6031530499458313),
  ('харьковчанин', 0.563445508480072),
  ('львовянин', 0.5659374594688416),
  ('москаль', 0.558702826499939),
  ('белорус', 0.5650672912597656),
  ('украинка', 0.5572485327720642),
  ('волынянин', 0.5463880896568298),
  ('русских', 0.5700317025184631

In [65]:
expanded_target_label_list = [item[0] for key in nearest_vals_dict for item in nearest_vals_dict[key]]

In [66]:
len(expanded_target_label_list)

187

In [67]:
expanded_target_label_list = list(set(expanded_target_label_list))

In [68]:
len(expanded_target_label_list)

139

In [69]:
import json

file_name = "/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/new_batch/expanded_target_labels.json"

with open(file_name, "w") as file:
    json.dump(expanded_target_label_list, file)

In [70]:
with open(file_name, "r") as file:
    loaded_unique_vals = json.load(file)

print(loaded_unique_vals)

['женщина', 'хохлятских', 'беженка', 'укровские', 'укропы', 'белорус', 'киевлянка', 'украинскую', 'малогосударствообразующие', 'харьковчанин', 'мерзавка', 'свинорылых', 'укронацистов', 'хохлятской', 'портников', 'украинской', 'кунис', 'пдрвля', 'украинское', 'свинособаки', 'бандерлоги', 'укропитеков', 'укрорейха', 'бабища', 'хлопчик', 'украинская', 'киевлянин', 'усср', 'чубатый', 'западенец', 'шмара', 'хлопци', 'хохла', 'укропской', 'жинка', 'вукраине', 'девка', 'карлица', 'щирый', 'укроп', 'наукраине', 'укры', 'хохлятский', 'мюмзики', 'укропов', 'наукраину', 'бацман', 'садура', 'белоруска', 'донбасс', 'украинскому', 'всук', 'зрадная', 'украинец', 'украина', 'пздц', 'бабёнка', 'свинособак', 'деваха', 'украинци', 'укр', 'насмикав', 'украинский', 'полячка', 'бандеровка', 'харьковчанка', 'долбаебы', 'немка', 'русских', 'украин', 'укропская', 'девушка', 'итальянка', 'поляк', 'украны', 'ницой', 'впольше', 'украинские', 'украинских', 'цханоская', 'укроканалы', 'лыжичко', 'всушный', 'хохлятск

In [65]:
# Do we need inflections?